In [ ]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import nltk
from keras.preprocessing import sequence
from keras.utils import to_categorical

In [ ]:
data = pd.read_csv('./data/tweets-mk.csv')

In [ ]:
data.columns = ['tweet', 'sentiment']

In [ ]:
data

,tweet,sentiment
0,имајте убав остаток од вечерва идам да пушам о...,1
1,rt therealpsmst therealpsmst за раст развој ин...,-1
2,топли кифли со премачкано сирење и топло кафе ...,1
3,rt tronovno слатка си ма у пичку матер,-1
4,јеби га aban ajramovic arno ras,-1
...,...,...
8560,остра реакција на германските пратеници ангела...,0
8561,marincepink кај нас и двете беа страшни,0
8562,falcon travel ви посакува убав продолжен викен...,0
8563,rt natetichka некои моменти се случуваат баш н...,0


In [ ]:
_stopwords = set(u"""
беше
еден
едно
јас
рече
сите
сум
ние
сме
овде
исто
така
на
и
во
се
го
не
да
е
што
од
со
за
или
тоа
ја
по
тој
но
а
ги
до
таа
му
кои
па
нив
ни
ќе
кој
итн
бил
кај
ова
врз
над
тие
a
две
би
она
неа
си
кое
ако
два
има
в
како
само
дека
една
туку
кога
сега
ми
потоа
низ
ти
кон
додека
веќе
нешто
него
уште
таму
ли
под
беа
ме
некој
ништо
тука
пред
им
каде
повторно
ниту
биде
толку
никогаш
мене
дали
тогаш
своите
сето
без
нема
околу
многу
полека
секогаш
зошто
те
добро
можеби
колку
можеше
нивните
преку
миг
знам
малку
вратата
ох
навистина
оваа
покрај
повеќе
овој
сеуште
имаше
својата
неговите
неговата
друго
зашто
немаше
воопшто
понекогаш
ах
зад
еднаш
својот
дури
себе
ви
токму
зарем
сте
било
сосема
секој
неговото
друг
мошне
ајде
можел
може
која
при
пак
сè
други
треба
ама
после
некоја
нас
бе
никој
одма
сѐ
ај
нѐ
неколку
вие
оние
мора
оди
еј
""".strip().split('\n'))

In [ ]:
stopwords = list(_stopwords)

In [ ]:
data['tweet'] = data['tweet'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))

In [ ]:
for i in range(0, len(data['tweet'])):
    data['tweet'][i] = re.sub('[a-z]', '', data['tweet'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data

,tweet,sentiment
0,имајте убав остаток вечерва идам пушам мрзнам ...,1
1,раст развој инвестиции кур поише нивоа слое...,-1
2,топли кифли премачкано сирење топло кафе млечн...,1
3,слатка ма у пичку матер,-1
4,јеби га,-1
...,...,...
8560,остра реакција германските пратеници ангела ме...,0
8561,двете страшни,0
8562,посакува убав продолжен викенд уживајте шетк...,0
8563,некои моменти случуваат баш право место прав...,0


In [ ]:
X = data['tweet']
Y = data['sentiment']

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 45)

In [ ]:
X_train

8416    киелце инста лајкнаа слика вардар дрес супер д...
2425                               нека  доцна пребирање 
6642                                      навивачки глупи
4202    емисии типот животните напаѓаат глупи луѓе ода...
87                        боли курот сања никулич бремена
                              ...                        
3704                                       глупави среќни
163     продолжуваат возбудувањата ракометната лига ша...
6012     разбрав та реченица контекст пуштање алудира ...
6558     цајканов прашува твитерџија шо презиваш делов...
7115                 глупа рекламата оне пакето шо барале
Name: tweet, Length: 6852, dtype: object

In [ ]:
with open('./data/vocab.txt') as f:
  vocab = f.readlines()

In [ ]:
vocab_size = len(vocab)
tokenizer = Tokenizer(num_words=vocab_size)

In [ ]:
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
words_to_index = tokenizer.word_index

In [ ]:
words_to_index

{'у': 1,
 'убаво': 2,
 'шо': 3,
 'убав': 4,
 'кур': 5,
 'убава': 6,
 'глуп': 7,
 'ден': 8,
 'ебам': 9,
 'глупости': 10,
 'ко': 11,
 'ебе': 12,
 'најубаво': 13,
 'пичка': 14,
 'убави': 15,
 'година': 16,
 'сакам': 17,
 'луѓе': 18,
 'време': 19,
 'глупи': 20,
 'имам': 21,
 'матер': 22,
 'тебе': 23,
 'поубаво': 24,
 'сеа': 25,
 'години': 26,
 'ебате': 27,
 'македонија': 28,
 'курва': 29,
 'одлично': 30,
 'курир': 31,
 'абе': 32,
 'коа': 33,
 'овие': 34,
 'животот': 35,
 'имаш': 36,
 'затоа': 37,
 'сакаш': 38,
 'значи': 39,
 'све': 40,
 'нека': 41,
 'некои': 42,
 'ве': 43,
 'боли': 44,
 'работи': 45,
 'твитер': 46,
 'денес': 47,
 'педер': 48,
 'пари': 49,
 'глупо': 50,
 'дома': 51,
 'скопје': 52,
 'еве': 53,
 'курац': 54,
 'цел': 55,
 'кафе': 56,
 'али': 57,
 'д': 58,
 'љубов': 59,
 'светот': 60,
 'живот': 61,
 'вака': 62,
 'работа': 63,
 'сам': 64,
 'вечер': 65,
 'брат': 66,
 'мислам': 67,
 'честитки': 68,
 'денот': 69,
 'дечко': 70,
 'јбг': 71,
 'биле': 72,
 'курот': 73,
 'фото': 74,
 'н

In [ ]:
max_words = 150
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('./data/vectors.txt')

maxLen = 150

In [ ]:
word_to_vec_map

{'на': array([-0.116834, -2.593129, -0.662719,  0.538977,  0.969747,  0.017023,
        -1.770179,  0.279806,  0.391856, -0.147258, -1.142896, -1.465371,
        -0.270436,  0.459887,  0.609936, -1.129635, -0.722509,  0.548455,
        -0.216557,  0.588572, -0.435363, -0.503655, -2.767345, -0.025689,
        -1.563052,  0.312015, -1.835009,  1.085421,  0.784335,  0.577316,
        -0.530903, -0.348741, -0.482355, -1.640338, -0.954254, -0.342151,
        -0.542551,  0.45253 ,  1.168054, -0.363318, -1.182208,  0.899547,
        -0.144695, -0.476123, -0.448368,  0.257229,  0.14596 , -0.337675,
         0.278088,  0.957149]),
 'и': array([ 0.42357 , -1.466927,  0.337187,  0.295284,  1.795702, -1.117347,
        -0.587998, -0.169289, -0.648504, -0.7555  , -1.139019, -0.552747,
         0.381161, -0.132394, -0.018727,  0.324348, -0.470822, -0.092994,
         0.476932, -0.208877, -1.765963,  0.612867, -2.49776 , -0.246616,
        -1.000035,  0.528951, -1.707778,  0.835224,  0.087325,  1.041

In [ ]:
vocab_len = len(words_to_index) + 1
embed_vector_len = word_to_vec_map['круг'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

In [ ]:
emb_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.044445,  0.579686,  0.343428, ..., -0.077814,  0.780707,
        -0.488163],
       [ 0.793862, -0.020569, -0.388085, ...,  0.086679,  0.276769,
        -0.504371],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.359785,  0.363385,  0.188449, ...,  0.10717 , -0.115232,
         0.11774 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])

In [ ]:
Y_train = to_categorical(Y_train, 3)
Y_test = to_categorical(Y_test, 3)

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=200, epochs=15)

Epoch 1/15
35/35 [==============================] - 77s 2s/step - loss: 0.9504 - accuracy: 0.5481
Epoch 2/15
35/35 [==============================] - 73s 2s/step - loss: 0.7101 - accuracy: 0.7035
Epoch 3/15
35/35 [==============================] - 72s 2s/step - loss: 0.6523 - accuracy: 0.7300
Epoch 4/15
35/35 [==============================] - 72s 2s/step - loss: 0.6119 - accuracy: 0.7521
Epoch 5/15
35/35 [==============================] - 73s 2s/step - loss: 0.5780 - accuracy: 0.7697
Epoch 6/15
35/35 [==============================] - 72s 2s/step - loss: 0.5467 - accuracy: 0.7918
Epoch 7/15
35/35 [==============================] - 73s 2s/step - loss: 0.5292 - accuracy: 0.7991
Epoch 8/15
35/35 [==============================] - 72s 2s/step - loss: 0.4834 - accuracy: 0.8203
Epoch 9/15
35/35 [==============================] - 73s 2s/step - loss: 0.4512 - accuracy: 0.8393
Epoch 10/15
35/35 [==============================] - 73s 2s/step - loss: 0.4516 - accuracy: 0.8369
Epoch 11/15
35/35 [

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test.argmax(1), y_pred.argmax(1)))

              precision    recall  f1-score   support

           0       0.87      0.50      0.63       278
           1       0.88      0.84      0.86       531
           2       0.81      0.93      0.86       904

    accuracy                           0.83      1713
   macro avg       0.85      0.76      0.79      1713
weighted avg       0.84      0.83      0.83      1713



In [ ]:
print(confusion_matrix(Y_test.argmax(1), y_pred.argmax(1)))

[[139  13 126]
 [  8 446  77]
 [ 13  48 843]]
